In [0]:
from pyspark.sql.functions import col, when

# 1. Leitura
df_alunos = spark.table("nttdataeducacao.silver.resultados_2024")
df_escolas = spark.table("nttdataeducacao.silver.escolas_2024")

# 2. Join
df_enrich = df_alunos.join(
    df_escolas,
    df_alunos.school_id == df_escolas.cod_entid,
    "left"
)

# 3. Seleção com a Correção de Tipo (Cast)
df_final = df_enrich.select(
    # Dados do Aluno
    df_alunos["*"],
    
    # Dados da Escola
    df_escolas["nome_entid"].alias("nome_escola"),
    df_escolas["dependencia"].alias("tipo_rede_censo"),
    df_escolas["localizacao"].alias("zona_escola"),
    
    # --- CORREÇÃO AQUI ---
    # Convertemos para 'double' antes de comparar com 7
    when(col("total_ensino_medio").cast("double") >= 7, "Sim") 
    .when(col("total_ensino_medio").cast("double") < 7, "Não")
    
    # Regra 3: Varredura de Nomes e Siglas
    .when(col("nome_entid").contains("INTEGRAL"), "Sim (Pelo Nome)")
    .when(col("nome_entid").contains("EEEMTI"), "Sim (Pelo Nome)") 
    .when(col("nome_entid").contains("EEMTI"), "Sim (Pelo Nome)")  
    .when(col("nome_entid").contains("CETI"), "Sim (Pelo Nome)")   
    .otherwise("Não / Não Informado").alias("possui_ensino_integral"),
    
    # Mantendo a hora para conferência (já convertida para número)
    df_escolas["total_ensino_medio"].cast("double").alias("media_horas_diarias")
)

# 4. Gravação
df_final.write.format("delta").mode("overwrite").saveAsTable("nttdataeducacao.gold.analytics_enem_completo")

display(df_final.limit(10))

In [0]:
from pyspark.sql.functions import col, when

df_resultados = spark.table("nttdataeducacao.gold.agg_ranking_escolas")
df_escolas = spark.table("nttdataeducacao.silver.escolas_2024")

df_join = df_resultados.join(
    df_escolas,
    df_resultados.school_id == df_escolas.cod_entid
)


df_check = df_join.withColumn(
    "tipo_enem_traduzido",
    when(col("school_type_id") == 1, "Federal")
    .when(col("school_type_id") == 2, "Estadual")
    .when(col("school_type_id") == 3, "Municipal")
    .when(col("school_type_id") == 4, "Privada")
    .otherwise("Outros")
).select(
    "school_id",
    "school_city",
    col("tipo_enem_traduzido").alias("O_que_o_ENEM_diz"),
    col("dependencia").alias("O_que_o_Censo_diz")
)

# 3. O Filtro da Mentira (Onde eles discordam)
df_divergencias = df_check.filter(
    col("O_que_o_ENEM_diz") != col("O_que_o_Censo_diz")
)

# Mostra o resultado
print(f"Total de escolas com divergência: {df_divergencias.count()}")
display(df_divergencias)

In [0]:
from pyspark.sql.functions import col, avg, round, count, desc

df_final = spark.table("nttdataeducacao.gold.analytics_enem_completo")
df_comparacao_final = df_final \
    .filter(col("school_id").isNotNull()) \
    .groupBy(
        "school_id",
        "nome_escola", 
        "school_state",
        "school_city"
    ) \
    .agg(
        count("candidate_id").alias("qtd_alunos"),
        avg("score_essay").alias("raw_redacao"),
        avg("score_math").alias("raw_matematica"),
        avg("score_languages").alias("raw_linguagens"),
        avg("score_humanities").alias("raw_humanas"),
        avg("score_nature").alias("raw_natureza")
    ) \
    .withColumn(
        "total_preciso", 
        col("raw_redacao") + col("raw_matematica") + 
        col("raw_linguagens") + col("raw_humanas") + col("raw_natureza")
    ) \
    .filter(col("qtd_alunos") >= 10) \
    .orderBy(desc("total_preciso")) \
    .limit(10) \
    .select(
        col("school_id"), 
        col("nome_escola"),
        col("school_state"),
        col("school_city"),
        round(col("total_preciso"), 2).alias("Nota_Total_Ranking"),
        round(col("raw_redacao"), 0).alias("Redacao"),
        round(col("raw_matematica"), 0).alias("Matematica"),
        round(col("raw_linguagens"), 0).alias("Linguagens"),
        round(col("raw_humanas"), 0).alias("Humanas"),
        round(col("raw_natureza"), 0).alias("Natureza")
    )

display(df_comparacao_final)

df_comparacao_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("nttdataeducacao.gold.agg_ranking_top10_final")

print("✅ Tabela salva com sucesso! Pode usar no Dashboard agora.")